In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

def scrape_website(url, word):
    driver = webdriver.Chrome()
    driver.get(url)

    if "livingspaces.com" in url:
        input_field = driver.find_element(By.ID, 'search')
        input_field.send_keys(word)
        form = input_field.find_element(By.XPATH, './ancestor::form')
        form.submit()
    elif "rcwilley.com" in url:
        input_field = driver.find_element(By.ID, 'searchBox')
        input_field.send_keys(word)
        submit_button = driver.find_element(By.ID, 'searchSubmit')
        submit_button.click()

    time.sleep(5)  
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    prices = []

    if "livingspaces.com" in url:
        product_items = soup.find_all('div', class_='product-item-container')
        for item in product_items:
            name_element = item.find('span', class_='name')
            price_element = item.find('span', class_='price')
            if name_element and price_element:
                name = name_element.text.strip()
                price = price_element.text.strip()
                prices.append({'name': name, 'price': float(price.replace('$', '').replace(',', ''))})

    elif "rcwilley.com" in url:
        product_items = soup.find_all('div', class_='productContent')
        for item in product_items:
            name_element = item.find('div', class_='productName')
            price_element = item.find('span', class_='price')
            if name_element and price_element:
                name = name_element.text.strip()
                price = price_element.text.strip()
                prices.append({'name': name, 'price': float(price.replace('$', '').replace(',', ''))})

    driver.quit()

    return prices

def calculate_average(prices):
    prices_float = [price['price'] for price in prices]
    return sum(prices_float) / len(prices_float) if prices_float else 0

def get_min_max(prices):
    if not prices:
        return None, None
    min_price = min(prices, key=lambda x: x['price'])
    max_price = max(prices, key=lambda x: x['price'])
    return min_price, max_price

def compare_prices(word):
    website1_url = 'https://www.livingspaces.com/'
    website2_url = 'https://www.rcwilley.com/'

    prices_website1 = scrape_website(website1_url, word)
    prices_website2 = scrape_website(website2_url, word)

    average_website1 = calculate_average(prices_website1)
    average_website2 = calculate_average(prices_website2)

    website1_name = website1_url.replace('https://www.', '').replace('.com/', '').capitalize()
    website2_name = website2_url.replace('https://www.', '').replace('.com/', '').capitalize()

    print(f"Item: {search_word}")
    print()

    # To print a list of all items on the first page.

    # print(f"Average price on {website1_name}: ${average_website1:.2f}")
    # for item in prices_website1:
    #     print(f"Name: {item['name']}, Price: {item['price']}")

    # print(f"\nAverage price on {website2_name}: ${average_website2:.2f}")
    # for item in prices_website2:
    #     print(f"Name: {item['name']}, Price: {item['price']}")


    print(f"Average price on {website1_name}: ${average_website1:.2f}")
    min_price, max_price = get_min_max(prices_website1)
    if min_price and max_price:
        print(f"Lowest price on {website1_name}: {min_price['name']} at ${min_price['price']:.2f}")
        print(f"Highest price on {website1_name}: {max_price['name']} at ${max_price['price']:.2f}")

    print(f"\nAverage price on {website2_name}: ${average_website2:.2f}")
    min_price, max_price = get_min_max(prices_website2)
    if min_price and max_price:
        print(f"Lowest price on {website2_name}: {min_price['name']} at ${min_price['price']:.2f}")
        print(f"Highest price on {website2_name}: {max_price['name']} at ${max_price['price']:.2f}")

    price_diff = abs(average_website1 - average_website2)
    print(f"\nPrice Comparison: {website1_name} is {'cheaper' if average_website1 < average_website2 else 'more expensive' if average_website1 > average_website2 else 'equally priced'} than {website2_name} by ${price_diff:.2f}")

search_word = input("Enter the word to search: ")
compare_prices(search_word)


Item: sectional

Average price on Livingspaces: $1743.75
Lowest price on Livingspaces: Calais Vanilla 112" 2 Piece Sectional With Right Arm Facing Chaise at $795.00
Highest price on Livingspaces: Vance Grey Leather 140" 6 Piece Zero Gravity Reclining Modular Sectional with Power Headrest, USB & Lumbar at $3995.00

Average price on Rcwilley: $2296.66
Lowest price on Rcwilley: Laguna Off-White Sofa-Chaise at $949.99
Highest price on Rcwilley: Denali Steel Gray 5 Piece Sectional at $4599.99

Price Comparison: Livingspaces is cheaper than Rcwilley by $552.91
